In [5]:
import pandas as pd
df=pd.read_parquet('C:/Users/Anna/Downloads/transaction_fraud_data.parquet')
df.head()

,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,is_card_present,device,channel,device_fingerprint,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,False,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,False,False,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,False,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,True,True,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,False,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,False,False,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,Australia,...,False,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,False,False,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,False,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,True,False,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [5]:
fraud_ratio=df['is_fraud'].mean()
fraud_ratio

0.19972818498066347

In [7]:
fraud_by_country = df[df['is_fraud']].groupby('country').size().reset_index(name='fraud_count')
top_fraud_countries = fraud_by_country.sort_values('fraud_count', ascending=False).head(10)

# Выводим результат
print(top_fraud_countries)

      country  fraud_count
8      Russia       299425
6      Mexico       298841
1      Brazil       298629
7     Nigeria       298600
0   Australia        37652
5       Japan        37592
3      France        37426
9   Singapore        37414
10         UK        37345
11        USA        37312


In [9]:
# Фильтруем только высокорисковых продавцов
high_risk_vendors = df[df['is_high_risk_vendor']]

# Считаем долю мошеннических транзакций среди них
fraud_ratio_high_risk = high_risk_vendors['is_fraud'].mean()
fraud_ratio_high_risk

0.19998632451602943

In [19]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Извлекаем дату и час (без минут и секунд)
df['timestamp'] = df['timestamp'].dt.floor('h')

# Группируем по клиенту и часу, считаем транзакции
transactions_per_client_hour = df.groupby(['customer_id', 'timestamp']).size()

# Выводим среднее количество транзакций клиента за час
average_transactions_per_hour = transactions_per_client_hour.mean()
average_transactions_per_hour

2.4527761909397174

In [25]:
avg_amount_by_city = df.groupby('city')['amount'].mean().reset_index(name='avg_transaction_amount')

# Сортируем по убыванию и берем топ-10 городов
top_cities = avg_amount_by_city.sort_values('avg_transaction_amount', ascending=False).head(10)
top_cities

,city,avg_transaction_amount
10,Unknown City,51315.780071
4,New York,568.872467
7,San Antonio,567.921894
9,San Jose,567.263760
6,Phoenix,565.730142
8,San Diego,565.593140
0,Chicago,565.277374
2,Houston,564.961388
3,Los Angeles,563.900769
5,Philadelphia,563.464431


In [27]:
fast_food_df = df[df['vendor_type'] == 'fast_food']

# Группируем по городу и считаем среднюю сумму транзакции
avg_amount_by_city = fast_food_df.groupby('city')['amount'].mean().reset_index(name='avg_transaction_amount')

# Сортируем по убыванию и берем топ-10 городов
top_cities = avg_amount_by_city.sort_values('avg_transaction_amount', ascending=False).head(10)
top_cities

,city,avg_transaction_amount
10,Unknown City,15214.407916
0,Chicago,264.450281
4,New York,263.296441
7,San Antonio,258.169691
3,Los Angeles,255.950324
8,San Diego,254.343884
1,Dallas,248.685716
9,San Jose,247.892502
2,Houston,246.913137
5,Philadelphia,243.440463


In [31]:
df_exchange=pd.read_parquet('C:/Users/Anna/Downloads/historical_currency_exchange.parquet')
df_exchange.head(5)

,date,AUD,BRL,CAD,EUR,GBP,JPY,MXN,NGN,RUB,SGD,USD
0,2024-09-30,1.443654,5.434649,1.351196,0.895591,0.747153,142.573268,19.694724,1668.736400,94.133735,1.280156,1
1,2024-10-01,1.442917,5.444170,1.352168,0.897557,0.746956,143.831429,19.667561,1670.694524,92.898519,1.284352,1
2,2024-10-02,1.449505,5.425444,1.348063,0.903056,0.752241,143.806861,19.606748,1669.653006,94.583198,1.286983,1
3,2024-10-03,1.456279,5.442044,1.351451,0.906018,0.754584,146.916773,19.457701,1670.097873,95.655442,1.294391,1
4,2024-10-04,1.460930,5.477788,1.355260,0.906452,0.761891,146.592323,19.363467,1649.763738,94.755337,1.296800,1


In [ ]:
# Преобразование дат в datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_exchange['date'] = pd.to_datetime(df_exchange['date'])

# Функция для конвертации в USD
def convert_to_usd(row):
    # 1. Находим соответствующую дату в таблице курсов
    rate_row = df_exchange[df_exchange['date'].dt.date == row['timestamp'].date()]
    
    if len(rate_row) == 0:
        return None  # Если курс на эту дату не найден
    
    # 2. Получаем курс для валюты транзакции
    currency = row['currency']
    if currency not in rate_row.columns:
        return None  # Если валюта не найдена в таблице курсов
    
    rate = rate_row[currency].values[0]
    
    # 3. Проверяем, что курс числовой и не NaN
    if pd.isna(rate) or not isinstance(rate, (int, float)):
        return None
    
    # 4. Умножаем сумму транзакции на курс
    return row['amount'] * rate

# Добавляем новый столбец
df['перевод в USD'] = df.apply(convert_to_usd, axis=1)


In [9]:
from ast import literal_eval

df = pd.concat([df, pd.json_normalize(df['last_hour_activity'])], axis=1)
df.head()

,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud,num_transactions,total_amount,unique_merchants,unique_countries,max_single_amount
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,False,False,False,"{'num_transactions': 1197, 'total_amount': 334...",False,1197,3.349856e+07,105,12,1.925481e+06
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,True,True,False,"{'num_transactions': 509, 'total_amount': 2011...",True,509,2.011476e+07,100,12,5.149117e+06
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,False,False,False,"{'num_transactions': 332, 'total_amount': 3916...",False,332,3.916385e+07,97,12,1.852242e+06
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,Australia,...,False,False,False,"{'num_transactions': 764, 'total_amount': 2201...",False,764,2.201260e+07,105,12,2.055798e+06
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,True,False,False,"{'num_transactions': 218, 'total_amount': 4827...",True,218,4.827636e+06,88,12,1.157231e+06


In [11]:
# Группируем по клиенту и вычисляем медиану
median_unique_merchants = df.groupby('customer_id')['unique_merchants'].median().reset_index()

# Переименовываем колонку для ясности
median_unique_merchants = median_unique_merchants.rename(columns={'unique_merchants': 'median_unique_merchants'})
median_unique_merchants.head()

,customer_id,median_unique_merchants
0,CUST_10000,93.0
1,CUST_10018,93.0
2,CUST_10022,95.0
3,CUST_10039,96.0
4,CUST_10102,98.0


In [13]:
# Находим 95-й квантиль по всем клиентам
threshold = median_unique_merchants['median_unique_merchants'].quantile(0.95)
high_activity_clients = median_unique_merchants[median_unique_merchants['median_unique_merchants'] > threshold]
print(f"95-й квантиль: {threshold:.2f}")
print(f"Количество клиентов выше порога: {len(high_activity_clients)}")

95-й квантиль: 100.00
Количество клиентов выше порога: 229


In [21]:
df_exchange=pd.read_parquet('C:/Users/Anna/Downloads/historical_currency_exchange.parquet')



KeyboardInterrupt: 

In [23]:
# Преобразование дат в datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_exchange['date'] = pd.to_datetime(df_exchange['date'])
# Извлекаем дату из timestamp (без времени) для сопоставления с exchange_rates
df['transaction_date'] = df['timestamp'].dt.date
df_exchange['exchange_date'] = df_exchange['date'].dt.date



In [27]:
valid_transactions = df[df['is_fraud'] == False]

MemoryError: Unable to allocate 5.71 MiB for an array with shape (5989047,) and data type bool

In [25]:
def get_exchange_rate(row):
    # Находим строку с нужной датой в таблице курсов
    rate_row = df_exchange[df_exchange['date'] == row['transaction_date']]
    currency = row['currency']
       
    return rate_row[currency].values[0]
# Добавляем курс валюты к каждой транзакции
df['exchange_rate'] = df.apply(get_exchange_rate, axis=1)
df.head()

MemoryError: Unable to allocate 856. MiB for an array with shape (15, 7483766) and data type object

In [ ]:
# Фильтрация нефродовых транзакций
